In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [11]:
# size_width_rugs_runners
# size_length_rugs_runners
# size_diameter_rugs_runners

# parameters
customer_id = '5'
formatted_attribute = 'backing_material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Rugs & Mats"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r"(?i)(\bbacking)|()"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [12]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [13]:
df = query_from_file(file_name='query/bbby.sql', params=params)
# df=df[df['value']=='n/a']
df

attribute  value    customers_name external_id  \
0      backing_material   Felt  bedbathandbeyond    45904578   
1      backing_material   Felt  bedbathandbeyond    45904844   
2      backing_material   Felt  bedbathandbeyond    45904943   
3      backing_material   Felt  bedbathandbeyond    45905032   
4      backing_material   Felt  bedbathandbeyond    60119629   
...                 ...    ...               ...         ...   
68206  backing_material  Vinyl  bedbathandbeyond    75983769   
68207  backing_material  Vinyl  bedbathandbeyond    75983784   
68208  backing_material  Vinyl  bedbathandbeyond    75983795   
68209  backing_material  Vinyl  bedbathandbeyond    76246291   
68210  backing_material  Vinyl  bedbathandbeyond    76477723   

                                                                      long_desc  \
0                       Kaleen Colinas Border 3-Foot x 5-Foot Area Rug in Ivory   
1                            Kaleen Colinas Diamonds 5' x 7'6 Area Rug in Camel   
2       Kaleen Colinas Solid 1-Foot 9-Inch x 2-Foot 10-Inch Accent Rug in Camel   
3                       Kaleen Colinas Solid 8-Foot x 10-Foot Area Rug in Camel   
4              Kaleen Mercery Lithograph 2-Foot x 3-Foot Accent Rug in Charcoal   
...                                                                         ...   
68206   Calloway Mills Home Sweet Home 17\" x 29\" Coir Door Mat in Green/Black   
68207         Calloway Mills Island Breeze 17\" x 29\" Multicolor Coir Door Mat   
68208   Calloway Mills Lobster Welcome 17\" x 29\" Coir Door Mat in Natural/Red   
68209      Calloway Mills Palm Tree Border 17\" x 29\" Multicolor Coir Door mat   
68210  Calloway Mills Welcome Aboard 17\" x 29\" Coir Door Mat in Natural/Black   

      resolution  
0       standard  
1       standard  
2       standard  
3       standard  
4       standard  
...          ...  
68206   standard  
68207   standard  
68208   standard  
68209   standard  
68210   standard  

[68211 rows x 6 columns]

In [14]:
# error

In [15]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

In [16]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [17]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()

In [18]:
# def re_extract(pattern, txt):
#     matches = re.findall(pattern, txt)
#     tmp_matches = []
#     for match in matches:
#         for tup in match:
#             if tup != '':
#                 tmp_matches.append(tup)
#     return list(set(tmp_matches))
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df

In [20]:
df=df[df['matches'].astype(str)!='[]']
df

attribute value    customers_name external_id  \
11229  backing_material   n/a  bedbathandbeyond    46423382   
11230  backing_material   n/a  bedbathandbeyond    46423399   
18676  backing_material   n/a  bedbathandbeyond    60759078   

                                                                long_desc  \
11229           Bath Carpet Ultra Tub Mat with Anti-Slip Backing in Clear   
11230  Bath Carpet Ultra Shower Stall Mat with Anti-Slip Backing in Clear   
18676         Bath Carpet Ultra Shower Mat with Anti-Slip Backing in Grey   

      resolution    matches  
11229   standard  [Backing]  
11230   standard  [Backing]  
18676   standard  [Backing]

In [ ]:
# df[150:200]

In [ ]:
# df=df[(df['matches'].astype(str)!='[]')]
# df

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
matches=df

In [ ]:
matches[curation_col]=matches['matches']
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)